<a href="https://colab.research.google.com/github/Sylver640/ADA-Informes/blob/main/Informe_Select.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Descripción del problema (selección)
**Entrada**: Conjunto de $n$ números diferentes $A$ más un índice $i$.

**Salida**: Elemento $x\in A$, que es mayor a exactamente $i-1$ elementos de $A$, siendo el elemento $x$ el estadístico de orden $i$-ésimo para el conjunto $A$.

Saliendo del área de optimización y orden, muchos algoritmos existen también para ser utilizados en aquella ciencia que estudia la variabilidad, organización o análisis de datos, es decir, **la estadística**. Un concepto de esta disciplina es el llamado **estadístico de orden**, que corresponde al $k$-ésimo valor más pequeño de una muestra. Aunque por su característica pueda parecer como algo poco utilizado, la realidad es que estos valores son una herramienta fundamental dentro de la estadística no paramétrica o de inferencia, en donde los datos no están distribuidos bajo un criterio especificado. Así, el problema de la **selección** está muy ligado a estos estadísticos, en donde manejamos la problemática de elegir el $i$-ésimo estadístico de orden en un conjunto de $n$ elementos.

![image](https://www.salamarkesa.com/wp-content/uploads/2016/11/apuntes-estadistica.jpg)

#2. Select

##2.1 Código
A continuación, el siguiente código presenta una implementación del algoritmo.

In [132]:
import random
from termcolor import cprint

comparisons = 0

def insertion_sort (array, verbose):
  if verbose == True: 
    print("\n#######")
    print("Se ordena un arreglo!")
    print("Arreglo a ordenar: ", array)

  #Contador de comparaciones en el ciclo for
  global comparisons

  #Se recorre el arreglo
  for i in range (1, len(array)):
    if verbose == True: print("Pasada",i)
    data = array[i]
    k = i-1
    if verbose == True:
      if array[k] < data: print("Elemento no movido\n")
    while k >= 0 and array[k] >= data:
      array[k+1] = array[k]
      k = k-1
      comparisons+=1
      if verbose == True: print(array,"\n")
    array[k+1] = data
    if verbose == True: print(f"Al finalizar pasada: {array}\n")

  if verbose == True: 
    print(f"Arreglo ordenado: {array}")
    print("#######\n")

  return array #Se retorna el arreglo ordenado

#Función que encuentra la mediana de un arreglo
def encontrarMediana(A, l, n, verbose):
  aux = [] #Arreglo auxiliar que guarda los elementos del subarreglo a analizar
  for i in range(l, l + n):
    aux.append(A[i])

  aux = insertion_sort(A, verbose) #Se ordena dicho subarreglo
  return aux[n // 2] #Como lo dice su nombre, se retorna el objeto que está justo al medio del arreglo, es decir, su mediana

#Implementación de partition que usa como pivote el último elemento
def partition(A, p, r, x, verbose):
  if verbose == True:
    print("\n#######")
    print("Se entra a función partition!")
  global comparisons
  
  #Se intercambia el valor x por el valor final, con tal de trabajar con este como pivote
  for i in range(p, r):
    if (A[i] == x):
      A[r], A[i] = A[i], A[r]
      break

  if verbose == True:
    print(f"Arreglo a dividir: {A[p:r+1]}")
  pivot = A[r] #Pivote es el último elemento
  if verbose == True:
    print(f"Pivote: [{A[r]}]")
  i = p #El puntero de intercambio comenzará desde el inicio del arreglo
  for j in range(p,r): #Mientras que el puntero j recorrerá todo el intervalo
    if A[j] <= pivot: #Si A[j] es menor o igual al pivote se entra a esta condicional if que realiza el intercambio
      if verbose == True:
        print(f"{A[j]} es menor a {pivot}! Swap!")
      A[j], A[i] = A[i], A[j] #Se intercambian las posiciones de A[j] y A[i] 
      i+=1 #Nuestro índice izquierdo aumenta una unidad, cerrando más el intervalo
      if verbose == True:
        print(f"Arreglo luego del intercambio: {A[p:r+1]}")
      comparisons+=1

  A[i], A[r] = A[r], A[i] #Finalmente, intercambiamos nuestro pivote con la posición A[i], dejándonos así un arreglo donde la mitad izquierda es menor
                          #y la mitad derecha es mayor al pivote
  comparisons+=1
  if verbose == True:
    print(f"Arreglo tras partition: {A[p:r+1]}")
    print("#######\n")
  return i #Retornamos como pivote la posición i de nuestro arreglo

def pivotSelection(A, l, r, verbose):
  n = r - l + 1 #Número de elementos
  medianas = [] #Lista que contiene las medianas de cada subarreglo

  i = 0
  while (i < n // 5): #Proceso de división del arreglo en subarreglos de cinco elementos
    medianas.append(encontrarMediana(A, l + i*5, 5, verbose)) #Se busca la mediana de este
    i+=1
  
  #Se busca la mediana de aquel último arreglo que tiene menos de cinco elementos
  if (i * 5 < n):
    medianas.append(encontrarMediana(A, l + i*5, n%5, verbose))
    i+=1
  
  if i == 1:
    medianaDeMedianas = medianas[i-1] #Si la lista solo contiene una mediana, será ésta la mediana de medianas
  else:
    medianaDeMedianas = pivotSelection(medianas, 0, i-1, verbose) #Llamada recursiva a pivotSelection con tal de encontrar la mediana de medianas
  
  return partition(A, l, r, medianaDeMedianas, verbose) #Finalmente, retorna el arreglo particionado con el pivote seleccionado

def select(A, l, r, k, verbose):
  #Tan solo se ingresará si k es menor o igual al número de elementos en el arreglo
  if (k > 0 and k <= r - l + 1):
    if verbose == True:
      print(f"Se busca pivote para el arreglo: {A[l:r+1]}")
    q = pivotSelection(A, l, r, verbose) #Se busca un pivote para particionar el arreglo

    #Si el pivote es igual a k, simplemente lo retornamos
    if (q - l ==  k - 1):
      return A[q]
    
    #Si la posición es mayor, recorremos recursivamente el subarreglo izquierdo
    if (q - l > k - 1):
      return select(A, l, q - 1, k, verbose)
    #Si no, lo hacemos por el subarreglo derecho
    return select (A, q+1, r, k - q + l - 1, verbose)

#Ejemplo
n = random.randint(1,6)
array = random.sample(range(1,100),n)
k = random.randint(1,n)

cprint(f"Arreglo de entrada: {array}", 'yellow', attrs=["bold"])
cprint(f"k = {k}", 'yellow')
element = select(array, 0, len(array)-1, k, verbose = False)
cprint(f"k-ésimo menor elemento del arreglo: {element}", "green", attrs=["bold"])
print(f"Número de comparaciones: {comparisons}")

Arreglo de entrada: [41, 62, 29, 14, 46, 94]
k = 4
k-ésimo menor elemento del arreglo: 46
Número de comparaciones: 21


##2.2 Descripción del algoritmo
El algoritmo **Select** es recursivo, llamándose a sí misma un total de dos veces. Éste recibe un arreglo o lista de $n$ elementos, su posición inicial, su posición final y un valor $k$, con el cual buscaremos el $k$-ésimo menor elemento dentro de la lista. A diferencia de **R-Select**, **Select** elige su pivote con ayuda de la función **pivotSelection**. En términos generales, la implementación de este algoritmo aplica la siguiente rutina gracias al algoritmo recién mencionado:
1. Al entrar a **pivotSelection**, el arreglo original de $n$ elementos se divide en $n/5$ grupos de 5 elementos cada uno, quedando el último grupo con menos elementos.
2. Luego, se encuentra la mediana de cada subarreglo, ordenándose con **Insertion Sort**.
3. Se aplica de forma recursiva **pivotSelection** con tal de encontrar la mediana de medianas ($m$) de las $n/5$ medianas identificadas anteriormente.
4. Así definimos a $m$ como el pivote para ser utilizado en la función **Partition**.
5. Finalmente, se aplicará recursivamente **Select** con el subarreglo izquierdo o derecho, sea cual sea el correspondiente para buscar el $k$-ésimo menor elemento.

##2.3 Ejemplo

##2.4 Ejecución del algoritmo paso a paso (`verbose = True`)
Al determinar que `verbose` sea igual a `True` en todas las funciones, al ejecutar el programa mostrarán cómo se realiza todo paso a paso, tal como se ve a continuación:

In [135]:
n = random.randint(1,6)
array = random.sample(range(1,100),n)
k = random.randint(1,n)
cprint(f"Arreglo de entrada: {array}; k = {k}\n", 'yellow', attrs=['bold'])
element = select(array, 0, len(array)-1, k, verbose = True)
cprint(f"k-ésimo menor elemento del arreglo: {element}", 'yellow', attrs=['bold'])

Arreglo de entrada: [35, 38, 65, 52]; k = 4

Se busca pivote para el arreglo: [35, 38, 65, 52]

#######
Se ordena un arreglo!
Arreglo a ordenar:  [35, 38, 65, 52]
Pasada 1
Elemento no movido

Al finalizar pasada: [35, 38, 65, 52]

Pasada 2
Elemento no movido

Al finalizar pasada: [35, 38, 65, 52]

Pasada 3
[35, 38, 65, 65] 

Al finalizar pasada: [35, 38, 52, 65]

Arreglo ordenado: [35, 38, 52, 65]
#######


#######
Se entra a función partition!
Arreglo a dividir: [35, 38, 65, 52]
Pivote: [52]
35 es menor a 52! Swap!
Arreglo luego del intercambio: [35, 38, 65, 52]
38 es menor a 52! Swap!
Arreglo luego del intercambio: [35, 38, 65, 52]
Arreglo tras partition: [35, 38, 52, 65]
#######

Se busca pivote para el arreglo: [65]

#######
Se ordena un arreglo!
Arreglo a ordenar:  [35, 38, 52, 65]
Pasada 1
Elemento no movido

Al finalizar pasada: [35, 38, 52, 65]

Pasada 2
Elemento no movido

Al finalizar pasada: [35, 38, 52, 65]

Pasada 3
Elemento no movido

Al finalizar pasada: [35, 38, 52, 65]

#3. Correctitud

#4. Tiempo de ejecución

#5. Experimentos